# Детектор спама

В современном обществе практически у каждого есть мобильный телефон, и все они регулярно получают сообщения (SMS/электронную почту) на свой телефон. Но главное, что большинство полученных сообщений будут спамом, и только несколько из них будут ненужными или необходимыми сообщениями. Мошенники создают мошеннические текстовые сообщения, чтобы обманом заставить вас предоставить им вашу личную информацию, такую ​​как ваш пароль, номер счета или номер социального страхования. Если у них есть такая информация, они могут получить доступ к вашей электронной почте, банковским или другим счетам.

Я буду использовать набор данных для обнаружения SMS-спама, который содержит текст SMS и соответствующую метку (нежелательное сообщение или спам).

Первым делом создадим и обучим модель которая будет определять сообщение являться спамом или нет. Для начала подключим нужные библиотеки для создания программы

In [4]:
import string

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


Также необходимо скачать 'стоп-слова'

In [5]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Теперь загрузим набор данных из kaggle для обучения модели и посмотрим из чего он состоит

In [6]:
df = pd.read_csv('spam_ham_dataset.csv')
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


Из этого набора данных нам потребуется только столбец text и label_num
И в столбце text есть элемент '\r\n' который нужно удалить

In [7]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))

Теперь проверим есть ли пропущенные значения в наборе данных

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


Так как в наборе данных нет пропущенных значений то можно идти дальше

Переведём весь текст в нижний регистр уберём пунктуацию и сделаем из него 'stem' версию, например

In [10]:
stemmer = PorterStemmer()
stemmer.stem('running')

'run'

In [12]:
stemmer = PorterStemmer()
stemmer.stem('sophisticated')

'sophist'

То есть 'stem' версия это основная часть слова

In [27]:
stemmer = PorterStemmer()
corpus = []

stopwords_set = set(stopwords.words('english'))

Нам нужен набор стопслов на английском, потому что набор данных содержит письма на английском языке

In [28]:
for i in range(len(df)):
    text = df['text'].iloc[i].lower() #перевод в нижний регистр
    text = text.translate(str.maketrans('', '', string.punctuation)).split() #убрали пунктуацию
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

Теперь сравним оригинал и очищенную версию

In [29]:
df.text.iloc[0]

"Subject: enron methanol ; meter # : 988291 this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary flow data provided by daren } . please override pop ' s daily volume { presently zero } to reflect daily activity you can obtain from gas control . this change is needed asap for economics purposes ."

In [30]:
corpus[0]

'subject enron methanol meter 988291 follow note gave monday 4 3 00 preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

Теперь векторизируем эти данные

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X[0]

array([1, 0, 0, ..., 0, 0, 0], shape=(42637,))

Теперь вместо текста У нас массив состоящий из чисел

И мы можем обучить модель

Для обучения модели я буду использовать RandomForestClasssifier

In [32]:
clf = RandomForestClassifier(n_jobs=-1)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9719806763285024

Обученная модель работает с точностью 97%

Теперь, чтобы пользователь мог проверить является ли письмо спамом, нужно перевести сообщение в нижний регистр, избавиться от пунктуации, сделать его stem версию и векторизировать

In [ ]:
email_to_classify = input()
email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()
email_text = [stemmer.stem(word) for word in text if word not in stopwords_set]
email_corpus = [email_text]

X_email = vectorizer.transform(email_corpus)
clf.predict(X_email)

Данный детектор спама сделал Камалеев Анвар, ученик из Казахстана г. Алматы